In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./projects/configs/co_dino/co_dino_5scale_lsj_swin_large_1x_coco.py') # faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py
root='../../../../dataset/' 

# dataset config 수정
cfg.data_root = root
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (128,128) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

print(cfg.data.test.pipeline[1]['img_scale'])

cfg.data.samples_per_gpu = 12
# cfg.data.workers_per_gpu = 2

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/co_dino_5scale_lsj_swin_large_3x_coco'
cfg.model.backbone.use_checkpoint = True
cfg.model.query_head.num_classes = 10
cfg.model.roi_head[0].bbox_head.num_classes = 10
cfg.model.bbox_head[0].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

(512, 512)


In [3]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


In [4]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [5]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2024-10-10 19:06:59,038 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-10-10 19:06:59,045 - mmcv - INFO - 
rpn_conv.weight - torch.Size([256, 256, 3, 3]): 
NormalInit: mean=0, std=0.01, bias=0 
 
2024-10-10 19:06:59,047 - mmcv - INFO - 
rpn_conv.bias - torch.Size([256]): 
NormalInit: mean=0, std=0.01, bias=0 
 
2024-10-10 19:06:59,047 - mmcv - INFO - 
rpn_cls.weight - torch.Size([9, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
2024-10-10 19:06:59,048 - mmcv - INFO - 
rpn_cls.bias - torch.Size([9]): 
NormalInit: mean=0, std=0.01, bias=0 
 
2024-10-10 19:06:59,049 - mmcv - INFO - 
rpn_reg.weight - torch.Size([36, 2

In [6]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters: {total_params}')

Total number of parameters: 234807819


In [7]:
# # 모든 레이어를 동결 (requires_grad를 False로 설정)
# for param in model.parameters():
#     param.requires_grad = False

# # 미세 조정을 위해 학습 가능하게 열 레이어 설정
# # backbone의 마지막 블록을 학습 가능하게 설정
# # for param in model.backbone.blocks[23].parameters():
# #     param.requires_grad = True

# # # RPN과 ROI head의 학습 가능하게 설정
# for param in model.rpn_head.parameters():
#     param.requires_grad = True

# for param in model.roi_head.parameters():
#     param.requires_grad = True

# for param in model.bbox_head.parameters():
#     param.requires_grad = True

# 학습 가능한 파라미터 확인
# trainable_params = [p for p in model.parameters() if p.requires_grad]
# print(f'학습 가능한 파라미터 수: {len(trainable_params)}')

In [ ]:
# 모델 학습
import torch
torch.cuda.empty_cache()
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-10-10 19:07:03,990 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-10-10 19:07:04,115 - mmdet - INFO - load checkpoint from local path: /data/ephemeral/home/taehan/test/level2-objectdetection-cv-18/Co-DETR/work_dirs/co_dino_5scale_lsj_swin_large_3x_coco/co_dino_5scale_lsj_swin_large_3x_coco.pth
2024-10-10 19:07:06,482 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for query_head.cls_branches.0.weight: copying a param with shape torch.Size([80, 256]) from checkpoint, the shape in current model is torch.Size([10, 256]).
size mismatch for query_head.cls_branches.0.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([10]).
size mismatch for query_head.cls_branches.1.weight: copying a param with shape torch.Size([80, 256]) from checkpoint, the shape in current model is torch.Size([10, 256]).
size mismatch for query_head.cls_branches.1.bias: copying a param w